In [1]:
%matplotlib inline
######################################################################################################
#
#         SCRIPT DESCRIPTION: Basic Q-Learning in a single intersection
#
#         Input: Map name, route file
#         Output: Tripinfo.xml
#
#####################################################################################################

# Set OPERATING SYSTEM
OS = "windows"
# Set MAP and time
used_map = 'Kings_X_positions'
data_tracking = True
# Set Path for Sumo Home folder
if OS == "iOS":
    sumo_home_dir = "/Users/acabrejasegea/sumo-0.32/"
elif OS == "windows":
    sumo_home_dir = "C:\Program Files (x86)\DLR\Sumo"
else:
    sys.exit("Please declare a supported operative system")

# Import basic libraries
import os
import sys
import optparse
import subprocess
import random
import random
import pdb
import xmltodict
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from __future__ import absolute_import
from __future__ import print_function
import json

# Current working directory
working_dir = os.getcwd()

# Set environment variable SUMO_HOME
try:
    # Use this line only to manually define SUMO_HOME path
    os.environ["SUMO_HOME"] = sumo_home_dir
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
except ImportError:
    sys.exit(
        "Please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

# Import API libraries
from sumolib import checkBinary
import traci

In [2]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
# Set important directories
if OS=="iOS":
    networks_dir = "/Users/acabrejasegea/sumo-0.32/Data/Networks/"
    scripts_dir = "/Users/acabrejasegea/sumo-0.32/tools/"
    runners_dir = "/Users/acabrejasegea/Desktop/ATI_Project/MLforFlowOptimisation/"
elif OS=="windows":
    networks_dir = "C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Networks"
    scripts_dir = "C:\\Program Files (x86)\\DLR\\Sumo\\tools\\"
    runners_dir = "C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Runners"
else:
    sys.exit("Please declare a supported operative system")
notebook = (os.path.splitext(notebook_name))[0]

In [5]:
positional_tracking = True
tracking_length = 3600

positional_data = [[] for _ in range(tracking_length)]

In [6]:
def run(learners = []):
    cycle_timer = 0
    timer = 0
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        timer = timer + 1
        
        if data_tracking == True:       
            vehicles = traci.vehicle.getIDList()
            if positional_tracking == True:
                for vehicle in vehicles:
                    positional_data[timer].append(tuple((vehicle, traci.vehicle.getPosition(vehicle),
                                                         traci.vehicle.getCO2Emission(vehicle), traci.vehicle.getNOxEmission(vehicle))))
    
    sys.stdout.flush()

### ORIGINAL SIMPLE CONTROLLER BELOW ###
# The program looks like this
#    <tlLogic id="0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="31" state="GrGr"/>
#        <phase duration="6"  state="yryr"/>
#        <phase duration="31" state="rGrG"/>
#        <phase duration="6"  state="ryry"/>
#    </tlLogic>
#
#def run():
#    """execute the TraCI control loop"""
#    step = 0
#    # we start with phase 2 where EW has green
#    traci.trafficlight.setPhase("0", 2)
#    while traci.simulation.getMinExpectedNumber() > 0:
#        traci.simulationStep()
#        if traci.trafficlight.getPhase("0") == 2:
#            # we are not already switching
#            if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
#                # there is a vehicle from the north, switch
#                traci.trafficlight.setPhase("0", 3)
#            else:
#                # otherwise try to keep green for EW
#                traci.trafficlight.setPhase("0", 2)
#        step += 1
#    traci.close()
#    sys.stdout.flush()

In [7]:
def get_options():
    optParser = optparse.OptionParser()
    optParser.add_option("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
    options, args = optParser.parse_args()
    return options

In [8]:
sumoBinary = checkBinary('sumo-gui')
# OR
#sumoBinary = checkBinary('sumo')

In [13]:
traci.start([sumoBinary, "-c", os.path.join(networks_dir, used_map,'osm.sumocfg'),
                             "--tripinfo-output", os.path.join((notebook + '_out'), 'tripinfo.xml')])

(17, 'SUMO 0.32.0')

In [14]:
run()

IndexError: list index out of range

In [15]:
## Exporting Positional data as JSON
with open(used_map+'_timesteps_'+str(tracking_length)+'.json', 'w') as outfile:
    json.dump(positional_data, outfile)

In [20]:
a = json.loads('C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Runners\\Kings_X_positions_timesteps_3600.json')
a

JSONDecodeError: Expecting value: line 1 column 1 (char 0)